In [4]:
!apt-get update
!apt-get install -y python3-pyaudio
!pip install pyaudio
!pip install google-cloud-speech
!pip install google-cloud-texttospeech
!pip install openai

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [920 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,552 k

In [9]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.40.0
    Uninstalling openai-1.40.0:
      Successfully uninstalled openai-1.40.0


In [4]:
import pyaudio
import wave
from google.cloud import speech_v1p1beta1 as speech
from google.cloud import texttospeech
import openai
import os

# Set up API keys and credentials
# Set up API keys and credentials
openai.api_key = 'sk-IBbcgJFY0n9ZsRLWn6_uje_HdvQE_bqWU7Sgy5S1iBT3BlbkFJd6IEqbyRnnI5sGH4yuxYkthJ4iRlvKRPjuvpLC3RMA'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/audio-project-431708-e1886fa0b670 (1).json"

# Initialize Google Speech-to-Text
speech_client = speech.SpeechClient()

# Initialize Google Text-to-Speech
tts_client = texttospeech.TextToSpeechClient()

# Audio configuration
RATE = 16000
CHUNK = 1024

def read_audio_file(file_path):
    with wave.open(file_path, 'rb') as wf:
        frames = []
        while True:
            data = wf.readframes(CHUNK)
            if not data:
                break
            frames.append(data)
        audio_data = b''.join(frames)
        # Logging captured audio data size
        print("Captured audio data of size:", len(audio_data))
        transcribe_audio(audio_data)

def transcribe_audio(audio_data):
    print("Transcribing audio data...")
    audio = speech.RecognitionAudio(content=audio_data)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code="en-US",
    )

    try:
        response = speech_client.recognize(config=config, audio=audio)
        for result in response.results:
            transcript = result.alternatives[0].transcript
            print(f"Transcript: {transcript}")
            process_with_chatgpt(transcript)
    except Exception as e:
        print(f"Error during transcription: {e}")

def process_with_chatgpt(transcript):
    print(f"Sending to ChatGPT: {transcript}")
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": transcript}
            ]
        )
        chatgpt_response = response.choices[0].message["content"].strip()
        print(f"ChatGPT Response: {chatgpt_response}")
        convert_text_to_speech(chatgpt_response)
    except Exception as e:
        print(f"Error during ChatGPT processing: {e}")
def convert_text_to_speech(text):
    # Logging that text-to-speech conversion is starting
    print(f"Converting text to speech: {text}")
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL)
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.LINEAR16)

    try:
        response = tts_client.synthesize_speech(input=synthesis_input, voice=voice, audio_config=audio_config)
        with open("output.wav", "wb") as out:
            out.write(response.audio_content)
            play_audio("output.wav")
    except Exception as e:
        print(f"Error during text-to-speech conversion: {e}")

def play_audio(file_path):
    wf = wave.open(file_path, 'rb')
    p = pyaudio.PyAudio()
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()), channels=wf.getnchannels(), rate=wf.getframerate(), output=True)

    data = wf.readframes(CHUNK)
    while len(data) > 0:
        stream.write(data)
        data = wf.readframes(CHUNK)

    stream.stop_stream()
    stream.close()
    p.terminate()
    print(f"Finished playing audio: {file_path}")

# Path to your audio file
audio_file_path = '/content/1003.wav'

# Read and process the audio file
read_audio_file(audio_file_path)


Captured audio data of size: 595200
Transcribing audio data...
Transcript: hi this is how are you I'm well thank you I just want to see how this car starts coming along
Sending to ChatGPT: hi this is how are you I'm well thank you I just want to see how this car starts coming along
Error during ChatGPT processing: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Transcript:  no
Sending to ChatGPT:  no
Error during ChatGPT processing: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Transcript:  I I another outside okay okay
Sending to ChatGPT:  I I another outside okay okay
Error during ChatGPT processing: You exceeded your current quota, please check your plan and billing details. For more information on